In [1]:
import pandas as pd
from tab_dataset import Cdataset
import ntv_pandas as npd

In [23]:
payload1 = pd.read_json("static_14h12m44s.json", orient="record")
payload2 = pd.read_json("static_14h12m46s.json", orient="record")
payload = pd.concat([payload1, payload2], ignore_index=True).reset_index()

In [8]:
payload = pd.read_json("bulk_evses_payload.json", orient="record").reset_index()
#payload

In [9]:
payload.columns

Index(['index', 'id_pdc_itinerance', 'id_pdc_local', 'id_station_local',
       'id_station_itinerance', 'nom_station', 'nom_amenageur', 'nom_enseigne',
       'contact_amenageur', 'siren_amenageur', 'nom_operateur',
       'contact_operateur', 'telephone_operateur', 'adresse_station',
       'code_insee_commune', 'coordonneesXY', 'gratuit', 'horaires',
       'condition_acces', 'implantation_station', 'restriction_gabarit',
       'station_deux_roues', 'tarification', 'nbre_pdc', 'cable_t2_attache',
       'prise_type_2', 'prise_type_autre', 'prise_type_chademo',
       'prise_type_combo_ccs', 'prise_type_ef', 'puissance_nominale',
       'accessibilite_pmr', 'paiement_acte', 'paiement_autre', 'paiement_cb',
       'reservation', 'observations', 'num_pdl', 'raccordement', 'date_maj',
       'date_mise_en_service'],
      dtype='object')

In [10]:
schema = {
    'relationships': [
         # relation unicité des pdl
         {"fields": ["id_pdc_itinerance", "index"],                    "link" : "coupled" },   
         # relations inter entités
         {"fields": ["id_station_itinerance", "nom_amenageur"],        "link" : "derived" },
         {"fields": ["id_station_itinerance", "siren_amenageur"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "contact_amenageur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_operateur"],        "link" : "derived" },
         {"fields": ["id_station_itinerance", "contact_operateur"],    "link" : "derived" },
         {"fields": ["id_station_itinerance", "telephone_operateur"],  "link" : "derived" },
         {"fields": ["id_station_itinerance", "nom_enseigne"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "coordonneesXY"],        "link" : "derived" },
         {"fields": ["id_pdc_itinerance", "id_station_itinerance"],    "link" : "derived" },
         # relations intra entité - station
         {"fields": ["id_station_itinerance", "nom_station"],          "link" : "derived" },
         {"fields": ["id_station_itinerance", "implantation_station"], "link" : "derived" },
         {"fields": ["id_station_itinerance", "date_maj"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "num_pdl"],              "link" : "derived" },
         {"fields": ["id_station_itinerance", "id_station_local"],     "link" : "derived" },
         {"fields": ["id_station_itinerance", "nbre_pdc"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "condition_acces"],      "link" : "derived" },
         {"fields": ["id_station_itinerance", "horaires"],             "link" : "derived" },
         {"fields": ["id_station_itinerance", "station_deux_roues"],   "link" : "derived" },
         {"fields": ["id_station_itinerance", "raccordement"],         "link" : "derived" },
         {"fields": ["id_station_itinerance", "date_mise_en_service"], "link" : "derived" },
         # relations intra entité - localisation
         {"fields": ["coordonneesXY", "adresse_station"],              "link" : "derived" },
         {"fields": ["coordonneesXY", "code_insee_commune"],           "link" : "derived" }
    ]
}

In [11]:
dic_errors = Cdataset(payload).check_relationship(schema)
dic_errors

{'index - id_pdc_itinerance': (),
 'nom_amenageur - id_station_itinerance': (),
 'siren_amenageur - id_station_itinerance': (),
 'contact_amenageur - id_station_itinerance': (),
 'nom_operateur - id_station_itinerance': (),
 'contact_operateur - id_station_itinerance': (),
 'telephone_operateur - id_station_itinerance': (),
 'nom_enseigne - id_station_itinerance': (),
 'coordonneesXY - id_station_itinerance': (),
 'id_station_itinerance - id_pdc_itinerance': (),
 'nom_station - id_station_itinerance': (),
 'implantation_station - id_station_itinerance': (),
 'date_maj - id_station_itinerance': (15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26),
 'num_pdl - id_station_itinerance': (),
 'id_station_local - id_station_itinerance': (),
 'nbre_pdc - id_station_itinerance': (),
 'condition_acces - id_station_itinerance': (),
 'horaires - id_station_itinerance': (),
 'station_deux_roues - id_station_itinerance': (),
 'raccordement - id_station_itinerance': (),
 'date_mise_

In [12]:
payload.loc[dic_errors['date_maj - id_station_itinerance'], ['date_maj', 'id_station_itinerance', 'id_pdc_itinerance']]

,date_maj,id_station_itinerance,id_pdc_itinerance
15,2025-08-06,FRS14PLM2921NT729329,FRS14ENFKC1
16,2025-08-05,FRS14PLM2921NT729329,FRS14ENFKC3
17,2025-08-05,FRS14PLM2921NT729329,FRS14ENFKC2
18,2025-08-05,FRS14PLM292C3UDJKANP,FRS14ENGBD1
19,2025-08-06,FRS14PLM292C3UDJKANP,FRS14ENGBD3
20,2025-08-06,FRS14PLM292C3UDJKANP,FRS14ENGBD2
21,2025-08-05,FRS14PLM292XSUPSC9QU,FRS14ENLJH1
22,2025-08-06,FRS14PLM292XSUPSC9QU,FRS14ENLJH2
23,2025-08-06,FRS14PLM292XSUPSC9QU,FRS14ENLJH3
24,2025-08-06,FRS14PLM2VQGDM1175EF,FRS14EFBWD1


In [13]:
payload.loc[dic_errors['date_mise_en_service - id_station_itinerance'], ['date_mise_en_service', 'id_station_itinerance', 'id_pdc_itinerance']]

,date_mise_en_service,id_station_itinerance,id_pdc_itinerance
0,2024-04-16,FRS14PLLV2DUR20R2BIR,FRS14EUKAL1
1,2024-04-16,FRS14PLLV2DUR20R2BIR,FRS14EUKAL2
14,2024-08-05,FRS14PLLV2DUR20R2BIR,FRS14EUKAL3
2,2024-04-16,FRS14PLLV2H0VQA296GE,FRS14EZZAA1
3,2024-04-16,FRS14PLLV2H0VQA296GE,FRS14EZZAA2
13,2024-08-05,FRS14PLLV2H0VQA296GE,FRS14EZZAA3
4,2024-04-19,FRS14PLLV6OREWV2DBTU,FRS14ENZWC1
5,2024-04-19,FRS14PLLV6OREWV2DBTU,FRS14ENZWC2
6,2024-06-11,FRS14PLLV6OREWV2DBTU,FRS14ENZWC3
